In [12]:
! pip install -r requirements.txt
! pip install -r /home/sojungkim2/legalmaster/LegalMaster/demo/requirements.txt

  Using cached transformers-4.29.1-py3-none-any.whl (7.1 MB)
  Using cached peft-0.3.0-py3-none-any.whl (56 kB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0.dev0
    Uninstalling transformers-4.31.0.dev0:
      Successfully uninstalled transformers-4.31.0.dev0
  Attempting uninstall: peft
    Found existing installation: peft 0.4.0.dev0
    Uninstalling peft-0.4.0.dev0:
      Successfully uninstalled peft-0.4.0.dev0
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-xq38_hel
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-xq38_hel
  Resolved https://github.com/huggingface/peft.git to commit 0b62b4378b4ce9367932c73540349da9a41bdea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-nf_o1dgw
  Run

In [1]:
import argparse
import logging
import random
import pickle
import tqdm
import shortuuid
import pandas as pd
import numpy as np
import os
import bitsandbytes as bn


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/sojungkim2/anaconda/envs/legalmaster/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
## models
import torch
import torch.nn as nn
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel
from utils.model import load_tokenizer_and_model

### Base 모델에 학습완료한 adapter를 merge하여 parameters 보기

##### Chat

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [4]:
base_model_path = '/home/sojungkim2/legalmaster/7Boutput'
#adapter_path = './adapter/'+['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
adapter_model_path = '/home/sojungkim2/legalmaster/LegalMaster/ChatAdapterTraining/adapter_chat'


In [5]:
tokenizer, model, _device = load_tokenizer_and_model(base_model_path, adapter_model_path, load_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('base_model.model.model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0096, -0.0301,  0.0085,  

In [7]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=4096, out_features=4096, bias=False
        

##### Legal

In [19]:
logger_1 = logging.getLogger(__name__)
logger_1.setLevel(logging.INFO)

In [20]:
base_model_path_1 = '/home/sojungkim2/legalmaster/7Boutput'
#adapter_path_1 = './adapter/'+['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
adapter_model_path_1 = '/home/sojungkim2/legalmaster/LegalMaster/LegalAdapterTraining/checkpoints_unmasked'


In [21]:
tokenizer_1, model_1, _device_1 = load_tokenizer_and_model(base_model_path_1, adapter_model_path_1, load_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
model_1.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('base_model.model.model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0096, -0.0301,  0.0085,  

### Chat + Legal

In [8]:
logger_2 = logging.getLogger(__name__)
logger_2.setLevel(logging.INFO)

In [9]:
base_model_path_2 = '/home/sojungkim2/legalmaster/MergeModels/base_chat'
#adapter_path_1 = './adapter/'+['llama_legal', 'llama_chat', 'llama_legal_chat', 'llama_chat_legal'][model_id]
adapter_model_path_2 = '/home/sojungkim2/legalmaster/LegalMaster/LegalAdapterTraining/checkpoints_unmasked'


In [10]:
tokenizer_2, model_2, _device_2 = load_tokenizer_and_model(base_model_path_2, adapter_model_path_2, load_8bit=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model_2.state_dict()

OrderedDict([('base_model.model.model.embed_tokens.weight',
              tensor([[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
                        5.9724e-05, -1.1957e-04],
                      [ 1.5289e-02, -1.2154e-02,  1.2512e-02,  ...,  1.3092e-02,
                        7.2174e-03, -6.8045e-04],
                      [ 1.7433e-03,  1.7633e-03, -1.4465e-02,  ..., -1.1444e-02,
                       -1.2665e-02,  3.7289e-04],
                      ...,
                      [-9.0179e-03,  3.0807e-02, -1.6708e-02,  ..., -1.2680e-02,
                        1.0437e-02,  4.2343e-03],
                      [-1.1368e-02, -1.4801e-02, -3.5667e-03,  ...,  6.5308e-03,
                       -2.2263e-02, -6.1455e-03],
                      [-1.3992e-02,  1.6985e-03, -2.1469e-02,  ...,  1.3527e-02,
                        2.8290e-02, -8.9111e-03]])),
             ('base_model.model.model.layers.0.self_attn.q_proj.weight',
              tensor([[-0.0096, -0.0301,  0.0085,  

In [12]:
model_2

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(
                in_features=4096, out_features=4096, bias=False
        